---
title: "Using iLINCS API Jupyter Notebook"
output:
  html_document:
    df_print: paged
  html_notebook: default
  word_document: default
  pdf_document: default
---

In [18]:
#install.packages(c('knitr', 'tinytex', 'httr', 'jsonlite', 'plotly', 'devtools', 'htmltools'))
# devtools::install_github('cmap/morpheus.R')
library(knitr)
# library(tinytex)
library(httr)
library(jsonlite)
library(plyr)
# library(data.table)
# library(plotly)
# library(htmltools)

## Display Signature Libraries

In [2]:
apiUrl <- "http://www.ilincs.org/api/SignatureLibraries"
req <- GET(apiUrl)
json <- httr::content(req, as = "text")
ilincs_libraries <- fromJSON(json)
ilincs_libraries[,c("libraryID","libraryName")]

libraryID,libraryName
LIB_1,Disease related signatures
LIB_10,Cancer therapeutics response signatures
LIB_11,LINCS gene overexpression signatures
LIB_12,DrugMatrix signatures
LIB_2,Connectivity Map signatures
LIB_3,ENCODE transcription factor binding signatures
LIB_5,LINCS chemical perturbagen signatures
LIB_6,LINCS consensus gene (CGS) knockdown signatures
LIB_8,LINCS RNA-Seq signatures
LIB_9,LINCS targeted proteomics signatures


## Searching for signature using BROAD ID

Searching for signature for given term. In this example "Diclofenac" coumpound will be shown as an example.

In [3]:
term <- "Diclofenac"
ilincs_libId<-"LIB_5"
apiUrl <- paste("http://www.ilincs.org/api/SignatureMeta/findTermWithSynonyms?term=",term,"&library=",ilincs_libId,sep="")
req <- GET(apiUrl)

ilincs_result_df<-fromJSON(httr::content(req,type="text"))$data
# head(ilincs_result_df)
head(ilincs_result_df[,c("cellline","compound","concentration","signatureid","is_exemplar")])

No encoding supplied: defaulting to UTF-8.


cellline,compound,concentration,signatureid,is_exemplar
A375,Diclofenac,10uM,LINCSCP_100,1
MCF7,Diclofenac,0.04uM,LINCSCP_135985,NA
HA1E,Diclofenac,10uM,LINCSCP_14684,0
PC3,Diclofenac,10uM,LINCSCP_154188,NA
PC3,Diclofenac,3.33uM,LINCSCP_154189,NA
A375,Diclofenac,10uM,LINCSCP_178714,0


## Searching for signature

###Searching for signature for given term. In this example "Diclofenac" coumpound will be shown as an example.


In [4]:
term <- "Diclofenac"
ilincs_libId<-"LIB_5"
apiUrl <- paste("http://www.ilincs.org/api/SignatureMeta/findTermWithSynonyms?term=",term,"&library=",ilincs_libId,sep="")
req <- GET(apiUrl)
ilincs_result_df<-fromJSON(httr::content(req,type="text"))$data
head(ilincs_result_df[,c("cellline","compound","concentration","signatureid","is_exemplar")])

No encoding supplied: defaulting to UTF-8.


cellline,compound,concentration,signatureid,is_exemplar
A375,Diclofenac,10uM,LINCSCP_100,1
MCF7,Diclofenac,0.04uM,LINCSCP_135985,NA
HA1E,Diclofenac,10uM,LINCSCP_14684,0
PC3,Diclofenac,10uM,LINCSCP_154188,NA
PC3,Diclofenac,3.33uM,LINCSCP_154189,NA
A375,Diclofenac,10uM,LINCSCP_178714,0


###Selecting a signature to analyze

In [5]:
ilincs_signatureId <- ilincs_result_df[1,"signatureid"]
ilincs_signatureId

[1] "LINCSCP_100"

## Getting signature data and vulcano plot
###The first step is to retrieve the session id for creation of the signature data table

In [6]:
req <- POST("http://www.ilincs.org/api/ilincsR/downloadSignature", body = list(sigID = paste(ilincs_signatureId)), encode = "json")
ilincs_sessionId<-unlist(content(req))
ilincs_sessionId

data 
"sig_Tue_Dec__4_22_44_55_2018_4557968"

### Retrieving data

In [7]:
fileUrl=paste("http://www.ilincs.org/tmp/",ilincs_sessionId,".xls",sep="")
signatureData<-read.table(fileUrl,sep="\t",header=T,stringsAsFactors = F)
head(signatureData)

signatureID,ID_geneid,Name_GeneSymbol,Value_LogDiffExp,Significance_pvalue
LINCSCP_100,16,AARS,1.047000,0.200843394
LINCSCP_100,23,ABCF1,0.787095,0.083514594
LINCSCP_100,25,ABL1,-2.933770,0.004530523
LINCSCP_100,30,ACAA1,2.821140,0.001994390
LINCSCP_100,39,ACAT2,0.528219,0.183911294
LINCSCP_100,47,ACLY,2.308770,0.049860369


### Insert volcano plot

In [8]:
apiUrl <- paste("http://www.ilincs.org/api/ilincsR/volcanoPlot?file=",ilincs_sessionId,".xls",sep="")
req <- GET(apiUrl)
ilincs_volcanoUrl<-paste("http://www.ilincs.org",content(req)$url,sep="")
include_graphics(ilincs_volcanoUrl)

[1] "http://www.ilincs.org/ocpu/tmp/x0257282d5b/graphics/1/png"
attr(,"class")
[1] "knit_image_paths" "knit_asis"       

%expand ${ }
${ ilincs_volcanoUrl }
<!-- ![alt text](http://www.ilincs.org/ocpu/tmp/x08c475ef57/graphics/1/png "Logo Title Text 1") -->
## Get connected signatures

In [9]:
apiUrl <- paste("http://www.ilincs.org/api/SignatureMeta/findConcordantSignatures?sigID=",ilincs_signatureId,"&lib=",ilincs_libId,sep="")
req <- GET(apiUrl)
# prettify(content(req,type="text"))
ilincs_conn_df<-fromJSON(httr::content(req,type="text"))
head(ilincs_conn_df)

No encoding supplied: defaulting to UTF-8.


signatureid,similarity,significance,nGenes,factor,compound,concentration,cellline,treatment,time,lincspertid,stitchID,perturbagenID,pubChemID,lincsSigID,is_exemplar
LINCSCP_176750,0.441378,0,978,NA,CHEMBL1208118,10uM,A375,CHEMBL1208118,6h,LSM-3598,NA,BRD-K71266197,25124826,CPC004_A375_6H:BRD-K71266197-236-01-9:10,1
LINCSCP_124,0.439186,0,978,NA,Erythromycin Estolate,10uM,A375,Erythromycin Estolate,6h,LSM-2876,CIDs00071277,BRD-K39746403,71277,CPC004_A375_6H:BRD-K39746403-084-03-1:10,1
LINCSCP_65,0.434709,0,978,NA,Chlorthalidone,10uM,A375,Chlorthalidone,6h,LSM-1417,CIDs00002732,BRD-A26384407,2732,CPC004_A375_6H:BRD-A26384407-001-15-2:10,1
LINCSCP_176580,0.424631,0,978,NA,Amidate,10uM,A375,Amidate,6h,LSM-1640,CIDs00036339,BRD-A54880345,36339,CPC004_A375_6H:BRD-A54880345-001-11-8:10,1
LINCSCP_176625,0.423807,0,978,NA,Piperidolate,10uM,A375,Piperidolate,6h,LSM-1951,CIDs00004839,BRD-A97479839,4839,CPC004_A375_6H:BRD-A97479839-003-08-3:10,1
LINCSCP_176698,0.409430,0,978,NA,TOCRIS-0930,10uM,A375,TOCRIS-0930,6h,LSM-2999,CIDs06604780,BRD-K45296539,6604780,CPC004_A375_6H:BRD-K45296539-003-01-0:10,0


## Find LINCS signatures connected to user a submitted signature
### Creating user submitted signature in the form of a gene list

In [10]:
top100signature <- signatureData[order(signatureData$Significance_pvalue)[1:100],]
head(top100signature)

,signatureID,ID_geneid,Name_GeneSymbol,Value_LogDiffExp,Significance_pvalue
379,LINCSCP_100,5993,RFX5,-10.00000,1.122981e-05
539,LINCSCP_100,9134,CCNE2,-6.00931,5.594009e-04
639,LINCSCP_100,10434,LYPLA1,-9.71956,7.459595e-04
831,LINCSCP_100,51056,LAP3,-9.71950,7.464972e-04
889,LINCSCP_100,55748,CNDP2,-9.71718,7.666483e-04
939,LINCSCP_100,79187,FSD1,-4.65059,8.272043e-04


### Uploading signature as a list genes for enrichment

In [19]:
apiUrl="http://www.ilincs.org/api/ilincsR/findConcordancesSC"
req <- POST(apiUrl, body = list(mode="geneList",metadata=TRUE,signatureProfile = list(genes=top100signature$Name_GeneSymbol)),encode = "json")
# ilincsEnrichedSignatures <- data.table::rbindlist(content(req)$sigScores, use.names = TRUE, fill = TRUE)
ilincsEnrichedSignatures <- ldply (content(req)$sigScores, data.frame)
head(ilincsEnrichedSignatures)

tableNames,zScores,compound,concentration,cellLine,time
LINCSCP_100,17.3629,Diclofenac,LSM-2160,10uM,A375
LINCSCP_65,7.4109,Chlorthalidone,LSM-1417,10uM,A375
LINCSCP_109,7.3279,RS-39604,LSM-2461,10uM,A375
LINCSCP_138,7.2192,Cilostazol,LSM-3491,10uM,A375
LINCSCP_66,6.8862,Altrenogest,LSM-1424,10uM,A375
LINCSCP_176623,6.7815,Dexamethasone Acetate,LSM-1915,10uM,A375


## Get GSEA plot

In [ ]:
apiUrl <- "http://www.ilincs.org/api/ilincsR/plotGSEA"
req <- POST(apiUrl, body = list(signatureId = ilincs_signatureId,genes = top100signature$ID_geneid, output="png"), encode = "json")
gseaPlotPng <- content(req)$fileName
gseaPlotUrl <- paste("http://www.ilincs.org/tmp/",gseaPlotPng,".png",sep="")
include_graphics(gseaPlotUrl)

## Signature as a list of up and down genes

In [ ]:
apiUrl="http://www.ilincs.org/api/ilincsR/findConcordancesSC"
topUpRegulatedGenes <- list(genesUp=top100signature$Name_GeneSymbol[top100signature$Value_LogDiffExp > 0])
topDownregulatedGenes <- list(genesDown=top100signature$Name_GeneSymbol[top100signature$Value_LogDiffExp < 0])
req <- POST("http://www.ilincs.org/api/ilincsR/findConcordancesSC", body = list(mode="UpDn",metadata=TRUE,signatureProfile = c(topUpRegulatedGenes, topDownregulatedGenes)),encode = "json")
ilincsUpDnConnectedSignatures <- data.table::rbindlist(content(req)$concordanceTable, use.names = TRUE, fill = TRUE)
head(ilincsUpDnConnectedSignatures)

## Group analysis of top 20 most connected signatures with signature of interest

In [ ]:
signatureGroup <-ilincsUpDnConnectedSignatures$signatureID[1:20]
apiUrl<-"http://www.ilincs.org/api/ilincsR/GroupLincsAnalysis"
req<-POST(apiUrl, body = list(idList = signatureGroup,noOfGenes = 50), encode = "json")                   
groupAnalysisSessionID <- content(req)$data[[2]]
groupAnalysisSessionID

### [Signature group heatmap on iLINCS](http://www.ilincs.org/apps/heatmap/?sessionID=`r groupAnalysisSessionID`&property=signatureID)

In [ ]:
heatmap_url<-paste("http://www.ilincs.org/apps/heatmap/?sessionID=",groupAnalysisSessionID,"&property=signatureID",sep = "")

### [Signature group PCA and tSNE app on iLINCS](http://www.ilincs.org/apps/pca/?sessionID=`r groupAnalysisSessionID`&property=signatureID)

In [ ]:
pca_url<-paste("http://www.ilincs.org/apps/pca/?sessionID=",groupAnalysisSessionID,"&property=signatureID",sep = "")

##Find connected signatures based on user submitted full signature
###Creating a file to upload from previouslsy downloaded signature file

In [ ]:
write.table(signatureData,file="sigFile.tsv",sep="\t",row.names=F,col.names = T,quote=F)
system(paste("head sigFile.tsv"))

###Upload the file

In [ ]:
sigFile <- "sigFile.tsv"
processFile <- content(POST("http://www.ilincs.org/api/SignatureMeta/upload", body=list(file=upload_file(sigFile))))
signatureFile <- processFile$status$fileName[[1]]
signatureFile

In [ ]:
apiUrl <- "http://www.ilincs.org/api/ilincsR/findConcordances"
req <- (POST(apiUrl, body = list(file=signatureFile, lib="LIB_5", path="/mnt/raid/tmp/"), encode = "form"))
output <- data.table::rbindlist(content(req)$concordanceTable, use.names = TRUE, fill = TRUE)
head(output)

## Find connected compound perturbations

In [ ]:
apiUrl <- paste("http://www.ilincs.org/api/ilincsR/signatureEnrichment?sigFile=",signatureFile,"&library=LIB_5&metadata=TRUE",sep="")
req <- GET(apiUrl)
json <- httr::content(req, as = "text")
iLincsConnectedCompoundPerturbations <- fromJSON(json)$enrichment
head(iLincsConnectedCompoundPerturbations)

## Get connected genetic perturbations

In [ ]:
apiUrl <- paste("http://www.ilincs.org/api/ilincsR/signatureEnrichment?sigFile=",signatureFile,"&library=LIB_6",sep="")
req <- GET(apiUrl)
json <- httr::content(req, as = "text")
iLincsConnectedGeneticPerturbations <- fromJSON(json)$enrichment
head(iLincsConnectedGeneticPerturbations)

## Retriving Signature Metadata

In [ ]:
apiUrl <- paste("http://www.ilincs.org/api/SignatureMeta/",ilincs_signatureId,sep="")
req <- GET(apiUrl)
l <- lapply(content(req), function(x) unlist(x))
ilincs_result <- data.frame(t(sapply(l,c)))
ilincsSigMetaData<-fromJSON(content(req,type="text"))
t(ilincs_result)

### List of LINCS Datasets

In [ ]:
apiUrl <- "http://www.ilincs.org/api/PublicDatasets/getDatasets?lincs=true"
req <- GET(apiUrl)
json <- httr::content(req, as = "text")
lincs_datasets <- fromJSON(json)
lincsDatasets <- data.frame(lincs_datasets$data)
lincsDatasets[c("experiment","assay","dataType")]

### Dataset metadata

**Example:** EDS-1014 

In [ ]:
experiment <- "EDS-1014"
apiUrl <- paste("http://www.ilincs.org/api/PublicDatasets/",experiment,sep="")
req <- GET(apiUrl)
json <- httr::content(req, as = "text")
datasetMetaData <- fromJSON(json)
datasetMetaData

[Heatmap for dataset `r experiment` on iLINCS](http://www.ilincs.org/apps/heatmap/?data_set=`r experiment`)

[PCA and tSNE for dataset `r experiment` on iLINCS](http://www.ilincs.org/apps/pca/?data_set=`r experiment`)

### Dataset sample metadata

In [ ]:
apiUrl <- paste("http://www.ilincs.org/api/ilincsR/getSamples?id=",experiment,sep="")
req <- GET(apiUrl)
json <- httr::content(req, as = "text")
sampleMeta <- fromJSON(json)
sampleMetaData <- data.frame(sampleMeta$data$rows)
head(sampleMetaData)

##Creating signature using iLINCS 
[Create signature from dataset `r experiment` on iLINCS](http://www.ilincs.org/ilincs/dataset/`r experiment`/createSignature)

##Creating signature using iLINCS API

In [ ]:
property <- "ER"
level1 <- "ER:+"
level2 <- "ER:-"
apiUrl <- "http://www.ilincs.org/api/ilincsR/LincsDataAnalysis"
req <- POST(apiUrl, body = list(exp = paste(experiment),prop = property,filterchk=paste(level1,level2,sep = ",,,"),includeORexclude=1), encode = "json")
createdSignaturesSessionID <- content(req)$sessionID
createdSignaturesSessionID

##Top 100 genes in created signature

In [ ]:
l <- lapply(content(req)$geneData, function(x) unlist(x))
ilincs_result <- data.frame(t(sapply(l,c)))
top100signatureData <- ilincs_result[1:100,c("Name_GeneSymbol","Value_LogDiffExp","Significance_pvalue")]
head(top100signatureData)

### Heatmap of data used in signature creation
### [Signature group heatmap on iLINCS](http://www.ilincs.org/apps/heatmap/?sessionID=`r createdSignaturesSessionID`&property=`r property`&geneCount=100)


In [14]:
heatmap_url2<-paste("http://www.ilincs.org/apps/heatmap/?sessionID=",createdSignaturesSessionID,"&property=",property,"&geneCount=100",sep = "")

ERROR: Error in paste("http://www.ilincs.org/apps/heatmap/?sessionID=", createdSignaturesSessionID, : object 'createdSignaturesSessionID' not found
